# Preparación de los datos para análisis y Machine Learning
En este notebook crearemos un *pipeline* de preprocesamiento de texto similar al visto al principio del estudio, pero más avanzado y haciendo uso de librerías como *spaCy* y *textacy*. Una vez completado, se obtendrá un texto limpio y tokenizado listo para su análisis.

Para este caso, se va a hacer uso del dataset creado en el apartado anterior, con más de 2000 comentarios del repoositorio *zigbee2mqtt*.

Al igual que en los cuadernos anteriores, comenzaremos cargando unos ajustes predefinidos para la ejecución del entorno virtual de python.

In [1]:
import sys, os

#Carga del archivo setup.py
%run -i ../pyenv_settings/setup.py

#Imports y configuraciones de gráficas
%run "$BASE_DIR/pyenv_settings/settings.py"

#Reset del entorno virtual al iniciar la ejecución
#%reset -f

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

# to print output of all statements and not just the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)

You are working on a local system.
Files will be searched relative to "..".


## Carga de los datos en Pandas
Cargaremos el dataset creado anteriormente con todos los comentarios de un repositorio de Github en Pandas, concretamente el archivo .csv (hay dos idénticos, uno en formato .csv y otro en .json)

In [2]:
#Ruta del archivo
file_path = "../data/output.csv"

#Carga del archivo en un DataFrame
df = pd.read_csv(file_path)

Antes de empezar a trabajar con los datos, revisaremos el nombre de las columnas y se cambiarán por otros nombres más genéricos en caso de considerarse necesario para una mejor coprensión y maniobrabilidad con el documento.

In [3]:
print(df.columns)

Index(['url', 'html_url', 'issue_url', 'id', 'node_id', 'user', 'created_at',
       'updated_at', 'author_association', 'body', 'reactions',
       'performed_via_github_app'],
      dtype='object')


Para el renombramiento de las columnas, definiremos un diccionario *column_mapping* en el que cada entrada corresponderá con el nombre de la columna original y el nuevo que se le dará. 

Si se considera que algunas columnas no son necesarias para el análisis, se pueden descartar nombrándolas como *None* o directamente sin incluirlas en el diccionario.

Viendo las columnas con las que cuenta el DataFrame se ve a simple vista que hay algunas columnas irrelevantes para el estudio, como las URLs, node_id, fechas de creación y actualización del post, asociaciones y la columna "performed_via_github_app". Estas serán descartadas a continuación sin incluirlas en el diccionario:

In [4]:
import ast #Para convertir cadenas JSON en objetos Python para la eliminación de campos innecesarios del campo 'user'

column_mapping = {
    'id' : 'id',
    'user' : 'user',
    'body' : 'text',
    'reactions' : None,
    'url' : None,
    'html_url' : None,
    'issue_url' : None,
    'node_id' : None,
    'created_at' : None,
    'update_at' : None,
    'author_association' : None,
    'performed_via_github_app' : None
}

#Se definen las columnas que se mantendrán
columns = [c for c in column_mapping.keys() if column_mapping[c] != None]

#Seleccionar y renombrar las columnas
df = df[columns].rename(columns=column_mapping)

#Normalizamos la columna user para extraer únicamente la información que interesa
# user_data = pd.json_normalize(df['user'])

# #Asegurar que las columnas que nos interesan existen
# if 'login' in user_data.columns and 'id' in user_data.columns:
#     df[['login', 'id']] = user_data[['login', 'id']]
# else:
#     raise ValueError("Las columnas 'login' e 'id' no se encuentran en los datos normalizados de 'user'")

# #Se elimina la columna 'user' original
# df = df.drop(columns=['user'])

#Muestra de una entrada para comprobar que se ha ejecutado correctamente
df.sample(1).T

,1811
id,1115280228
user,"{'login': 'Tanjohnson99', 'id': 97446509, 'node_id': 'U_kgDOBc7qbQ', 'avatar_url': 'https://private-avatars.githubusercontent.com/u/97446509?jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaX..."
text,"> Fixed for me (Chrome) by opening **dev tools**, right clicking on the **reload button**, and selecting **""Empty Cache and Hard Reload""**\r\n\r\nWorking for me like this ! Thanks"


## Guardado y carga de un Data Frame
Para guardar el Data Frame en disco se hará uso de una base de datos SQL utilizando SQLite. No es necesario contar con conocimientos avanzados de SQL pues se hará uso de la librería *sqlite3* de python que integra todas las funciones necesarias para trabajar con este tipo de bases de datos, como mucho, se usarán sentencias SQL básicas para realizar acciones sobre la base de datos..

Cuando guardamos el Data Frame en la base de datos, no se almacena el índice del Data Frame, y todos los datos ya existentes son sobreescritos.

In [5]:
#sqlite3 ya está importado en el archivo settings.py cargado al iniciar el programa

#Damos nombre a la DB, nos conectamos a ella, guardamos los datos, y se cierra conexión
db_name = "zigbee2mqtt_comments.db"
con = sqlite3.connect(db_name)
df.to_sql("comments", con, index=False, if_exists="replace")
con.close()

2678

El Data Frame se lee de forma muy sencilla:

In [6]:
con = sqlite3.connect(db_name)
df = pd.read_sql("select * from comments", con)

## Limpieza de los datos
Antes de la tokenización de los datos es necesario limpiar los datos recopilados de ruido innecesario y distintos formatos en el texto. Algunos ejemplos de estos pueden ser los caracteres especiales, URLs incluidas en los comentarios, etiquetas, emoticonos, etc.

Para esta función se usarán expresiones regulares junto con la librería *regex* para detectar y eliminar todos estos elementos innecesarios para el posterior análisis.

In [7]:
#import re -> importado en settings.py

RE_SUSPICIOUS = re.compile(r'[&#<>{}\[\]\\]') #símbolos sospechosos de introducir ruido

def impurity(text, min_len=10): #se ignoran textos de menos de 10 caracteres
    if text == None or len(text) < min_len:
        return 0
    else:
        return len(RE_SUSPICIOUS.findall(text))/len(text)

Ahora se procederá a depurar y eliminar el ruido en los comentarios de los posts extraídos del repositorio con el que hemos estado trabajando hasta este momento,

In [8]:
#Se añade la columna "impurity" al Data Frame que mostrará el porcentaje de cada comentario
df['impurity'] = df['text'].progress_apply(impurity, min_len=10)

#Algunas muestras de los registros con más ruido
df[['text', 'impurity']].sort_values(by='impurity', ascending=False).head(5)

100%|██████████| 2678/2678 [00:00<00:00, 119613.93it/s]


,text,impurity
1877,> If I put this config in my zigbee2mqtt 1.25.0-1 and also the below in the config folder it doesn't work. \n> \n> \n> \n> data_path: /config/zigbee2mqtt\n> \n> socat:\n> \n> enabled: false\n> \...,0.08
1902,"Sure, done in #307.",0.05
1866,"> > Thanks mate. Gonna check this tomorrow in the afternoon to (hopefully) help you out. I'm now gonna take a nap.\n> \n> \n> \n> Mate i found the problem.\n> \n> if you reset to default values, t...",0.05
1884,> It seems to me that within the configuration tab of the 'Zigbee2MQTT' addon the following lines need to be present (and the rest removed):\n> \n> ```\n> \n> data_path: /config/zigbee2mqtt\n> \n>...,0.05
1446,"That works and shows correct state in HA.\r\n\r\nJust FYI - I forgot to mention before that `{{ value_json.tilt }}` needs to be ""{{ value_json.tilt }}"" to work.",0.05


Como se observa, el grado de impurezas no es demasiado elevado, pero si se omiten facilitará el trabajo el análisis, ademaś de que siempre se tiene que tener en cuenta debido a que se está trabajando con contenido generado por usuarios, y este puede ser un caso excepcional en el que no hay demasiado ruido.

### Conteo de otras posibles palabras que pueden introducir ruido
Se importará la función *count_words* utilizada en *1-textual_data* para realizar el conteo de palabras de otras etiquetas que no se han tenido en cuenta y que también pueden introducir ruido.

In [14]:
from collections import Counter

def count_words(df, column='tokens', preprocess=None, min_freq=2):

    #procesa los tokens y actualiza el contador
    def update(doc):
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(tokens)

    #crea el contador y recorre todos los datos
    counter = Counter()
    df[column].progress_map(update)

    #transforma el contador a dataframe
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    freq_df = freq_df.query('freq >= @min_freq')
    freq_df.index.name = 'token'
    
    return freq_df.sort_values('freq', ascending=False)

count_words(df, column='text', preprocess=lambda t: re.findall(r'<[\w/]*>', t))

100%|██████████| 2678/2678 [00:00<00:00, 109349.16it/s]


,freq
token,
<anonymous>,51
<redacted>,5
</details>,5
<details>,5
</summary>,5
<summary>,5
<REDACTED>,4
<template>,4
</script>,4


## Eliminación de ruido con Expresiones Regulares
Se va a crear una función que definirá una serie de expresiones regulares que serán utilizadas para detectar en el texto una serie de patrones que cumplen aquellas palabras susceptibles de introducir ruido. Estas serán sustituidas por texto plano o eliminadas directamente del texto.

In [15]:
import html

def clean(text):
    # convert html escapes like &amp; to characters.
    text = html.unescape(text) 
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)
    # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

Ahora se aplicará esta función a la columna "text" del Data Frame que almacena los comentarios de los usuarios en el repositorio, además, se añadirá una nueva columna con el texto limpio, de modo que se pueda visualizar más fácilmente los cambios entre el texto original y el texto sin ruido.

In [21]:
df['clean_text'] = df['text'].progress_apply(clean)

#Muestras de la columna "clean_text"
print(df[['text', 'clean_text']].sample(5))

100%|██████████| 2678/2678 [00:00<00:00, 17652.01it/s]

                                                                                                                                                                                                         text  \
1208  Same problem since I installed my GLEDOPTO GL-C-009P today. \n\n```Exception in handle_state_message_received when handling msg on 'zigbee2mqtt/Toilet': '{"brightness":102,"last_seen":"2022-12-05T...   
476   I had to recreate the configuration.yaml in the Zigbee2mqtt folder. I first created a copie and then added all the devices to the new file. I had to remove references to groups and or devices.yaml...   
290                                                                     This issue is stale because it has been open 30 days with no activity. Remove stale label or comment or this will be closed in 7 days   
1112  BTW: the same hardware setup on the same base system Debian 11 but with homebridge 1.6. on board (zigbee2mqtt 1.29.2 and module homebridge-z2m 1.9.2) works pe

Ya se había indicado antes que con suerte la información extraída no contenía demasiado ruido, pese a eso, se puede ver a simple vista la diferencia entre algunos textos originales y los textos ya sin ruido.

## Normalización de caracteres con *textacy*
Caracteres especiales como los acentos, apóstrofes, diéresis, etc. pueden ser un problema a la hora de tokenizar un texto, por ello se normalizará sustituyendo estos caracteres por equivalentes ASCII para evitar así inconvenientes.

Se utilizará la librería *textacy* creada para trabajar junto con *spaCy*.

In [26]:
import textacy
import textacy.preprocessing as tprep

# En caso de que se cuente con una versión menor a la 0.11
if textacy.__version__ < '0.11':
    def normalize(text):
        text = tprep.normalize_hyphenated_words(text)
        text = tprep.normalize_quotation_marks(text)
        text = tprep.normaliza_unicode(text)
        text = tprep.remove_accents(text)

        return text
    
else:
    #En mi caso cuento con la versión 0.13
    def normalize(text):
        text = tprep.normalize.hyphenated_words(text)
        text = tprep.normalize.quotation_marks(text)
        text = tprep.normalize.unicode(text)
        text = tprep.remove.accents(text)
        text = tprep.replace.urls(text)
        text = tprep.replace.emails(text)
        text = tprep.replace.emojis(text)
        
        return text

En este caso, se aplicará la normalización sobre el texto limpio sin ruido obtenido en el apartado anterior.

In [27]:
df['normalized_text'] = df['clean_text'].progress_apply(normalize)

#Muestras de la columna "clean_text"
print(df[['text', 'clean_text', 'normalized_text']].sample(5))

100%|██████████| 2678/2678 [00:00<00:00, 2692.82it/s]

                                                                                                                                                                                                         text  \
322   Same result for 1.35.1-1 version backup and 1,5 year backwards even. Now I'm wondering where the device data should be if it is not in the addon's backup file.\r\n\r\nOnly these files are in the b...   
2663                                                                                                                                                  Thanks for the report. This indeed also fixed my issue.   
2356  > I have no idea how you guys got it working but no matter what I do, every time I start this addon it overwrites the /config/zigbee2mqtt/configuration.yaml file with its default content.\r\n> \r\...   
1936       Maybe try a reinstall of Zigbee2mqtt?\r\nI had your issue, till I did a full upgrade of core and OS. Then I found I had the deCONZ addon still running, t